In [1]:
from finetune import Workspace as fW
from pretrain import Workspace as pW
import os
import hydra
from omegaconf import OmegaConf
import numpy as np

/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


In [2]:
import pathlib
import hydra
hydra.core.global_hydra.GlobalHydra.instance().clear()
config_dir = pathlib.Path('./')
hydra.initialize(config_path=config_dir)

hydra.initialize()

In [3]:
cfg = hydra.compose(config_name='finetune.yaml', overrides=[])
f_workspace = fW(cfg)

workspace: /home/seolho/workplace/self-supervised-rl/for commit/cic


In [4]:
'''
skill = np.zeros(64).astype(np.float32)
f_workspace.eval(skill)
'''

| eval  | F: 0 | S: 0 | E: 0 | L: 1000 | R: 193.4848 | T: 0:00:00


In [4]:
cfg = hydra.compose(config_name='pretrain.yaml', overrides=[])
p_workspace = pW(cfg)

workspace: /home/seolho/workplace/self-supervised-rl/for commit/cic


In [5]:
cfg

{'agent': {'_target_': 'agent.cic.CICAgent', 'name': 'cic', 'reward_free': '${reward_free}', 'obs_type': 'states', 'obs_shape': [24], 'action_shape': [6], 'device': '${device}', 'lr': 0.0001, 'critic_target_tau': 0.01, 'update_every_steps': 2, 'use_tb': '${use_tb}', 'use_wandb': '${use_wandb}', 'num_expl_steps': 4000, 'hidden_dim': 1024, 'feature_dim': 1024, 'stddev_schedule': 0.2, 'stddev_clip': 0.3, 'skill_dim': 64, 'scale': 1.0, 'update_skill_every_step': 50, 'nstep': 3, 'batch_size': 128, 'project_skill': True, 'init_critic': True, 'rew_type': 'og', 'update_rep': True, 'temp': 0.5}, 'reward_free': True, 'domain': 'walker', 'obs_type': 'states', 'frame_stack': 3, 'action_repeat': 1, 'discount': 0.99, 'num_train_frames': 2000010, 'num_seed_frames': 4000, 'eval_every_frames': 10000, 'num_eval_episodes': 10, 'snapshots': [100000, 500000, 1000000, 2000000], 'snapshot_dir': '../../../pretrained_models/${obs_type}/${domain}/${agent.name}/${experiment}', 'replay_buffer_size': 1000000, 'rep

In [6]:
#np.random.uniform(0,1,64).astype(np.float32)
skill = np.zeros(64).astype(np.float32)
#skill = np.random.uniform(0,1,64).astype(np.float32)
#workspace.eval(skill)

f_workspace.gather_trajectories(skill, 10000)
len(f_workspace.replay_storage)

#p_workspace.agent.behavior_cloning(f_workspace.replay_iter, 0)


12050

In [8]:
skill = np.zeros(64).astype(np.float32)
p_workspace.eval(skill)

| eval  | F: 0 | S: 0 | E: 0 | L: 1000 | R: 19.3694 | T: 0:00:00


In [9]:
for i in range(100):
    #skill = np.random.uniform(0,1,64).astype(np.float32)
    skill = np.zeros(64).astype(np.float32)
    #workspace.eval(skill)
    
    f_workspace.gather_trajectories(skill, 1000)
    len(f_workspace.replay_storage)

    print(p_workspace.agent.behavior_cloning(f_workspace.replay_iter, 0))

{'batch_reward': 0.1391632854938507, 'bc_loss': 0.4016333818435669}
{'batch_reward': 0.14747251570224762, 'bc_loss': 0.3905754089355469}
{'batch_reward': 0.14643533527851105, 'bc_loss': 0.38684481382369995}
{'batch_reward': 0.16485702991485596, 'bc_loss': 0.3821464478969574}
{'batch_reward': 0.14301274716854095, 'bc_loss': 0.3958275020122528}
{'batch_reward': 0.13627636432647705, 'bc_loss': 0.3756754398345947}
{'batch_reward': 0.14457392692565918, 'bc_loss': 0.37170514464378357}
{'batch_reward': 0.1434091031551361, 'bc_loss': 0.40581852197647095}
{'batch_reward': 0.14798255264759064, 'bc_loss': 0.38474246859550476}
{'batch_reward': 0.14991886913776398, 'bc_loss': 0.39812788367271423}
{'batch_reward': 0.14860355854034424, 'bc_loss': 0.3977634906768799}
{'batch_reward': 0.14146144688129425, 'bc_loss': 0.3843810558319092}
{'batch_reward': 0.14360158145427704, 'bc_loss': 0.39050421118736267}
{'batch_reward': 0.14936241507530212, 'bc_loss': 0.39078444242477417}
{'batch_reward': 0.1650961041

In [10]:
skill = np.zeros(64).astype(np.float32)
p_workspace.eval(skill)

| eval  | F: 0 | S: 0 | E: 0 | L: 1000 | R: 21.4710 | T: 0:00:00


In [11]:
class Until:
    def __init__(self, until, action_repeat=1):
        self._until = until
        self._action_repeat = action_repeat

    def __call__(self, step):
        if self._until is None:
            return True
        until = self._until // self._action_repeat
        return step < until

In [12]:
t = Until(10)

In [ ]:
    def behavior_cloning(self, replay_iter, step):
        metrics = dict()

        if step % self.update_every_steps != 0:
            return metrics

        batch = next(replay_iter)

        obs, action, reward, discount, next_obs, skill = utils.to_torch(
            batch, self.device)

        with torch.no_grad():
            obs = self.aug_and_encode(obs)
        
            next_obs = self.aug_and_encode(next_obs)


        if self.use_tb or self.use_wandb:
            metrics['batch_reward'] = reward.mean().item()

        # extend observations with skill
        obs = torch.cat([obs, skill], dim=1)
        next_obs = torch.cat([next_obs, skill], dim=1)
        
        stddev = utils.schedule(self.stddev_schedule, step)
        dist = self.actor(obs, stddev)
        agent_action = dist.sample(clip=self.stddev_clip)
        
        bc_loss = torch.mean((action - agent_action)**2)
        
        self.actor_opt.zero_grad(set_to_none=True)
        loss.backward()
        self.actor_opt.step()
        # update actor
        metrics['bc_loss'] = bc_loss.item()

        return metrics
